## Vector Clock implementation using Python 3
A vector clock is an algorithm for generating a partial ordering of events in a distributed system and detecting causality violations. [reference]

In a distributed system consist of N processes each process is going to have a(n) vector(array) of N logical clocks(simply counters), one clock per process; The values of these counters will be updated by the following rules:
* Initialize all logical clocks to zero in each vector.
* Each time a process experiences an internal event, it increments its own logical clock in the vector by one.
* Each time a process sends a message, it increments its own logical clock in the vector by one and then piggyback a copy of its own vector to the message.
* Each time a process receives a message, it increments its own logical clock in the vector by one and updates each element in its vector by taking the maximum of the value in its own vector clock and the value in the vector in the received message.

[reference]: https://en.wikipedia.org/wiki/Vector_clock

### Implementation Details:
* Instead of using an array of N counters a dictionary (map) is used in which the dictionary key is the process id and the value of dictionary represent the logical clock counter. This way there is no longer need to define the number of processes a head and we could have variable number of processes.
* Every event takes two argument an arbitrary name (which should be uniqe for that class) and duration, The duration is updating the actual cpu counter of the process (the real clock in each event)
* Send method also is implemented like an event so it has thoes two arguments that event has(name is replaced with message-msg- here plus it also implements the receive operation as well so it takes the receiver process as an extra argument
* The receive method is implied in the send which actually make sens (There should be a send event happening for the receive side to take place)
* Colon ':' character shouldn't be used for process or event name
* Each event name consist of two parts first the process id that the event is happening, a colon as a seprator and unique name of that event in that process

In [1]:
class process(object):
    
    events=dict() # Static dictionary variable for process class to store all events and messages
    
    def __init__(self, name):
        self.id = name
        self.vClocks=dict()
        self.vClocks[self.id]=0 # initialize its logical clock to zero
        self.timer = 0
        
    def event(self, name, duration=30):
        self.vClocks[self.id]+=1
        self.timer += duration
        process.events[self.id+':'+name] = self.vClocks.copy()

    # send and receive events both are happening in this method
    # the message is going to be the name of send event for process 1 and receive event process 2
    def send(self, msg, receiver, duration=10):
        # send event
        self.vClocks[self.id]+=1
        self.timer += duration
        process.events[self.id+':'+msg] = self.vClocks.copy()
        # send happens before receive
        # receive event
        receiver.vClocks[receiver.id]+=1
        for key,val in self.vClocks.items():
            if key in receiver.vClocks:
                receiver.vClocks[key] = max(receiver.vClocks[key], val)
            else:
                receiver.vClocks[key] = val
        receiver.timer += duration
        process.events[receiver.id+':'+msg] = receiver.vClocks.copy()
        
    def printVectorClocks(self):
        print("Process:",self.id,"timer:", self.timer)
        for key,val in sorted(self.vClocks.items()):
            print(key,"->",val)
        print("=======================")
        
    
    def printEventVectorClock(self, eventName):
        name = self.id+':'+eventName
        print("Event:",name)
        for key,val in sorted(process.events[name].items()):
            print(key,"->",val)
        print("=======================")
    
    # This method will return true if event1 is happening before event2 of xternalProcess
    @staticmethod
    def HappendBefore(event1, event2):
        vc1 = process.events[event1] # vector clock for event1
        vc2 = process.events[event2] # vector clock for event2
        
        # There should be at least one strict smaller value in vector clocks of event1 and no bigger value to make
        # the whole event happening before event2
        strictSamllerFound = False 
        
        # Those items that are not in vc1 have zero value(samller) in vc1, so there is no need to
        # check them agian
        for key, val in vc2.items(): 
                if key not in vc1 and val > 0:
                   strictSamllerFound = True
                
        for key, val in vc1.items(): 
            if key in vc2:
                if vc2[key] < val:
                    return False
                elif val < vc2[key]:
                    strictSamllerFound = True
            elif val > 0:
                return False
            
        return strictSamllerFound
    
    
    @staticmethod
    def PrintPartialOrder(eventName):
        causes = []
        effects = []
        concurrents = []
        for key in process.events:
            if process.HappendBefore(key, eventName):
                causes.append(key)   
            elif process.HappendBefore(eventName, key):
                effects.append(key)
            else :
                concurrents.append(key)
                
        
        print("Causes: ", end='')
        for t in causes :
            print(t, " ", end='')
            
        print("\nEffects: ", end='')
        for t in effects :
            print(t, " ", end='')
            
        print("\nConcurrents: ", end='')
        for t in concurrents :
            print(t, " ", end='')
        

### Here is an implementation example of vector clocks for events shown in the following figure from wikipedia.
![caption](https://upload.wikimedia.org/wikipedia/commons/5/55/Vector_Clock.svg)

In [2]:
A = process(name="A")
B = process(name="B")
C = process(name="C")

C.send("cb", B, 20)
B.send("ba", A, 30)
B.send("bc1", C, 23)
A.send("ab", B, 14) # a.k.a B receives ab (after sending bc1 to c)
C.send("ca1", A, 17)
B.send("bc2", C, 30)
C.send("ca2", A, 21)


A.printVectorClocks()
B.printVectorClocks()
C.printVectorClocks()

Process: A timer: 82
A -> 4
B -> 5
C -> 5
Process: B timer: 117
A -> 2
B -> 5
C -> 1
Process: C timer: 111
A -> 2
B -> 5
C -> 5


You could print the vector clock for an event or test if an event happened before the other one.

In [3]:
A.printEventVectorClock("ab")
print("Event cb of C is happning before event ab of A:", process.HappendBefore(C.id+":cb", A.id+":ab"))
print("Event bc2 of B is happning before event bc1 of C:", process.HappendBefore(B.id+":bc2", C.id+":bc1"))

Event: A:ab
A -> 2
B -> 2
C -> 1
Event cb of C is happning before event ab of A: True
Event bc2 of B is happning before event bc1 of C: False


You could also print all the causes and effects and concurrent events for a given event

In [4]:
process.PrintPartialOrder(B.id+":ab")

Causes: C:cb  B:cb  B:ba  A:ba  B:bc1  A:ab  
Effects: B:bc2  C:bc2  C:ca2  A:ca2  
Concurrents: C:bc1  B:ab  C:ca1  A:ca1  